In [1]:
"""4 consecutive years of grouped by playerID """
import csv
from ipywidgets import widgets
from IPython.display import display

In [2]:
fname = "..//transform-1_aggregate-stats-alt_cole//batters_with_four_seasons_of_ten_hits.csv"
#fname_out = "Career_Batting1.csv"

In [3]:
#Indices 
pidx, yrx, gx, abx, rx, hx, dblx, tplx = 0, 1, 2, 3, 4, 5, 6, 7
hrx, rbix, sbx, csx, bbx, = 8, 9, 10, 11, 12
sox,ibbx, hbpx, shx,sfx = 13,14,15,16,17


In [4]:
class PlayerSeasonStats:
    def __init__(self, pid):
        self.pid = pid
        self.attribution = [] #A list :[[yearid,interest],....]

    def add_data(self, line, interested_attribution):
        self.attribution.append([line[yrx], line[interested_attribution]])

    def years_asc(self, include_first=True, include_last=True):
        self.attribution.sort()
        if (len(self.attribution) < 4):
            print("Not enough stats: " + self.pid)
            while len(self.attribution) < 4:
                self.attribution.append(None, None)
        
        i_start = 0 if include_first else 1
        i_end = len(self.attribution) if include_last else len(self.attribution) - 1
        sets_no = max(i_end - i_start - 3, 0)
        return [self.attribution[i_start + i: i_start + i + 4] for i in range(sets_no)]

In [5]:
def readcsv(fname):
    '''Returns a list of lines from the file'''
    f = open(fname, 'r')
    reader = csv.reader(f)
    lines = list(reader)
    f.close()
    return lines

In [6]:
def writecsv(fname, season_stats,
             include_first_season=True,
             include_last_season=True):
    header = ["playerID", "yearID", "season1",
             "season2","season3","season4"]
    outlines = [header]
    for pid in season_stats:
        year_seasons = season_stats[pid].years_asc(include_first=include_first_season,
                                                   include_last=include_last_season)
        for year_season in year_seasons:
            line = [pid, # playerID
                    year_season[0][0], # season 1 yearID
                    year_season[0][1], # season 1 
                    year_season[1][1], # season 2
                    year_season[2][1], # season 3
                    year_season[3][1]] # season 4
            outlines.append(line)
    f = open(fname, 'w', newline = '')
    writer = csv.writer(f)
    writer.writerows(outlines)
    f.close()
    print("output successfully, file name is ", fname)  #Number of lines written to file"""

In [7]:
def build_name_dict(fname):
    if(statistic.value =="G"):
        interested_attribution = gx
    elif(statistic.value =="AB"):
        interested_attribution = abx
    elif(statistic.value =="R"):
        interested_attribution = rx
    elif(statistic.value =="H"):
        interested_attribution = hx
    elif(statistic.value =="2B"):
        interested_attribution = dblx
    elif(statistic.value =="3B"):
        interested_attribution = tplx
    elif(statistic.value =="HR"):
        interested_attribution = hrx
    elif(statistic.value =="RBI"):
        interested_attribution = rbix
    elif(statistic.value =="SB"):
        interested_attribution = sbx
    elif(statistic.value =="CS"):
        interested_attribution = csx
    elif(statistic.value =="BB"):
        interested_attribution = bbx
    elif(statistic.value =="SO"):
        interested_attribution = sox
    elif(statistic.value =="IBB"):
        interested_attribution = ibbx
    elif(statistic.value =="HBP"):
        interested_attribution = hbpx
    elif(statistic.value =="SH"):
        interested_attribution = shx
    elif(statistic.value =="SF"):
        interested_attribution = sfx
    d = {}
    lines = readcsv(fname)
    for row in lines[1:]:
        pid = row[pidx]
        if pid not in d:
            d[pid] = PlayerSeasonStats(pid)
        d[pid].add_data(row, interested_attribution)
        
    return d

In [8]:
'''
    Build GUI, Choose the attribution wanted 
'''

statistic = widgets.RadioButtons(options = ["G", "AB", "R","H","2B", "3B","HR","RBI","SB", 
                                           "CS","BB", "SO","IBB","HBP","SH","SF"], value = "G",
                     description = "Attribute:")
display(statistic)

include_first = widgets.Checkbox(value=True, description = "Include Player's first season")
include_last = widgets.Checkbox(value=True, description = "Include Player's last season")
display(include_first)
display(include_last)

filename_suffix_input = widgets.Text(description = "File suffix:")
display(filename_suffix_input)


button = widgets.Button(description = "Create CSV")
display(button)
filename_display = widgets.Label()
display(filename_display)


def get_filename():
    suff = ""
    if (filename_suffix_input.value):
        suff = filename_suffix_input.value + "_"
    return "last_four_years_playerID_" + suff + str(statistic.value) + ".csv"

def on_input_change(_=None):
    filename_display.value = get_filename()

def on_button_clicked(b):
    lines = readcsv(fname)
    names_dict = build_name_dict(fname)
    fname_out = get_filename()
    #name = list(names_dict.keys())
    #name.sort()
    writecsv(fname_out, names_dict,
             include_first_season=include_first.value,
             include_last_season=include_last.value)

filename_suffix_input.observe(on_input_change, names='value')
statistic.observe(on_input_change, names='value')
button.on_click(on_button_clicked)
# trigger filename display:
on_input_change()